"""
参考URL
https://github.com/salu133445/ismir2019tutorial/blob/main/musegan.ipynb
https://salu133445.github.io/pypianoroll/read_write.html
midi→楽器ごとのmidi化はmidi_to_independent_instrument_midi.py で行えます。
"""

In [ ]:
# !pip3 install torch matplotlib tqdm ipywidgets livelossplot gdown "pypianoroll>=1.0.2"

from ipywidgets import interact, IntSlider
import os
import os.path
from pathlib import Path
import pypianoroll
from pypianoroll import Multitrack, Track

In [ ]:
#midiファイル保存用のディレクトリを作成
lakh_midi_dir="lakh_midi_dir"
if not os.path.isdir(lakh_midi_dir):
        os.mkdir(lakh_midi_dir)

In [ ]:
#dataフォルダを作成後その配下にlack pianoroll dataset(npz形式)を保存
!mkdir -p data

!gdown -O data/lpd_5_cleansed.tar.gz https://drive.google.com/uc?id=1yz0Ma-6cWTl6mhkrLnAVJ7RNzlQRypQ5
!gdown -O data/id_lists_amg.tar.gz https://drive.google.com/uc?id=1hp9b_g1hu_dkP4u8h46iqHeWMaUoI07R
!gdown -O data/id_lists_lastfm.tar.gz https://drive.google.com/uc?id=1mpsoxU2fU1AjKopkcQ8Q8V6wYmVPbnPO

!tar zxf data/lpd_5_cleansed.tar.gz -C data/
!tar zxf data/id_lists_amg.tar.gz -C data/
!tar zxf data/id_lists_lastfm.tar.gz -C data/

amgフォルダ:楽曲ジャンルごとに楽曲idリストが書かれているtxtファイルが入っているフォルダ

全txtを参照しているので、lpd_5_cleansedの全曲のリストがid

ジャンルごとに分けたいなら、amg以下の階層でpath渡せば何とかなると思います。

In [ ]:
dataset_root = Path("data/lpd_5/lpd_5_cleansed/")
id_list = []

for path in os.listdir("data/amg"):
    filepath = os.path.join("data/amg", path)
    if os.path.isfile(filepath):
        with open(filepath) as f:
            id_list.extend([line.rstrip() for line in f])
id_list = list(set(id_list))

In [ ]:
def msd_id_to_dirs(msd_id):
    """Given an MSD ID, generate the path prefix.
    E.g. TRABCD12345678 -> A/B/C/TRABCD12345678"""
    return os.path.join(msd_id[2], msd_id[3], msd_id[4], msd_id)

id_listの中のidを、上のmsd_id_to_dirsで変換 

該当する箇所にあるファイルをmidi形式に変換

id名でlakh_midi_dir配下にファイル保存

In [ ]:
for id in id_list: 
    song_dir = dataset_root / msd_id_to_dirs(id)
    multitrack = pypianoroll.load(song_dir / os.listdir(song_dir)[0])
    pypianoroll.write(path=lakh_midi_dir+"/"+id+".mid",multitrack=multitrack)
#11th Gen Intel(R) Core(TM) i7-11700 @ 2.50GHzで36分くらいかかります。